In [1]:
import pandas as pd
import numpy as np
import mysql.connector as mariadb
from astropy.time import Time
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import gridspec
import time
from ipywidgets import *

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
pd.set_option('display.max_columns',120)

In [4]:
con = mariadb.connect(user='ztf', database='ztf')

In [5]:
def plot_lightcurve(dflc_lm, Classtar = 0., Thrsh_RB = 0., days_ago=True, g=True, r=False, i=False):
    
    dflc = pd.read_sql_query("SELECT objectId, jd, magpsf, sigmapsf, diffmaglim ,ra, decl, fid, \
        classtar, rb, candid, programid FROM alerts where objectId='{}' ORDER BY jd".format(dflc_lm), con=con)
    
    dflm = pd.read_sql_query("SELECT objectId, jd, diffmaglim, fid FROM alerts_limmag where objectId='{}' \
        ORDER BY jd".format(dflc_lm), con=con)
    
    dflc = dflc[dflc.classtar > Classtar]
    dflc = dflc[dflc.rb > Thrsh_RB]    
    
    filter_color1 = filter_color2 = filter_color3 = {}
    if g:
        filter_color1 = {1:'green'}

    if r:
        filter_color2 = {2:'red'}

    if i:
        filter_color3 = {3:'blue'}

    filter_color = {**filter_color1, **filter_color2, **filter_color3}


    plt.figure(figsize=(12,5))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 

    ax0 = plt.subplot((gs[0]))
    
    if days_ago:
        now = Time.now().jd
        tc = dflc.jd - now
        tm = dflm.jd - now
        xlabel = 'Days Ago'
    else:
        tc = dflc.jd
        tm = dflm.jd
        xlabel = 'Time (JD)'
    
    for fid, color in filter_color.items():
        
        # plotting detections
        v = (dflc.fid == fid)
        if np.sum(v):
            plt.errorbar(tc[v],dflc.loc[v,'magpsf'], dflc.loc[v,'sigmapsf'],fmt='.',color=color,alpha=0.5)
            
        # plotting upper-limits from the prv_candidate
        #w = (dflm.fid == fid)
        #if np.sum(w):
        #    plt.scatter(tm[w],dflm.loc[w,'diffmaglim'], marker='v',color=color,alpha=0.25)
    


    plt.gca().invert_yaxis()
    plt.xlabel(xlabel)
    plt.ylabel('Magnitude')
    
    
    ax1 = plt.subplot((gs[1]))
    ax1.plot((dflc.ra - (dflc.ra.mean()))*3600, (dflc.decl - (dflc.decl.mean()))*3600,'bo',ms=2)

    plt.xlabel(r'$\Delta RA \, (arcsec)$')
    plt.ylabel(r'$\Delta Dec \, (arcsec)$')

In [6]:
interact(plot_lightcurve, dflc_lm='ZTF18aagsenq')

interactive(children=(Text(value='ZTF18aagsenq', description='dflc_lm'), FloatSlider(value=0.0, description='C…

<function __main__.plot_lightcurve(dflc_lm, Classtar=0.0, Thrsh_RB=0.0, days_ago=True, g=True, r=False, i=False)>